In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

def load_and_format(in_path):
    out_df = pd.read_json(in_path)  
    out_images = out_df.apply(lambda c_row: [np.stack([c_row['band_1'],c_row['band_2'],np.add(c_row['band_1'],c_row['band_2'])/2], -1).reshape((75,75,3))],1)
    
    out_images = np.stack(out_images).squeeze()
    out_images[:,:,:,0] = (out_images[:,:,:,0]-out_images[:,:,:,0].min())/(out_images[:,:,:,0].max()-out_images[:,:,:,0].min())   #train_df['band_1'].apply(iso)
    out_images[:,:,:,1] = (out_images[:,:,:,1]-out_images[:,:,:,1].min())/(out_images[:,:,:,1].max()-out_images[:,:,:,1].min())    # ,np.add(c_row['band_1'],c_row['band_2'])/2]
    out_images[:,:,:,2] = (out_images[:,:,:,2]-out_images[:,:,:,2].min())/(out_images[:,:,:,2].max()-out_images[:,:,:,2].min())
    return out_df, out_images#[:,:,:,:2]
train_df,train_images = load_and_format('data/train.json')
test_df,test_images = load_and_format('data/test.json')[:5000]

In [2]:
import scipy
from sklearn.model_selection import train_test_split
train_images_76 = np.array([scipy.misc.imresize(img,(72,72)) for img in train_images])
test_images_76 = np.array([scipy.misc.imresize(img,(72,72)) for img in test_images])
X_train = train_images_76    #, X_test = train_test_split(train_images_76, test_size=0.33, random_state=42)
y_train = train_df['is_iceberg'].values.reshape(-1,1)  #, y_test = train_test_split(train_df['is_iceberg'].values.reshape(-1,1), test_size=0.33, random_state=42)
full_train = np.append(X_train,test_images_76,axis = 0)

In [3]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape
from keras.models import Model
from keras.optimizers import Adam

import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf


inputs = Input(shape=(72,72,3))
x = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)
encoded_flat = Flatten()(encoded)
####autoencoder
h = Conv2D(8, (3, 3), activation="relu", padding="same")(encoded)
h = UpSampling2D((2, 2))(h)
h = Conv2D(16, (3, 3), activation="relu", padding="same")(h)
h = UpSampling2D((2, 2))(h)
h = Conv2D(32, (3, 3), padding="same")(h)
h = UpSampling2D((2, 2))(h)
outputs = Conv2D(3,(3,3), activation='relu', padding='same')(h)

####standard neural network

autoencoder = Model(input=inputs, output=outputs)
print(autoencoder.summary())
autoencoder.compile(optimizer='adam', loss='mse')


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 72, 72, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 16)        448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 36, 36, 8)         1160      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 8)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 8)         584       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 9, 9, 8)           0         
__________

/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:30: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [9]:
autoencoder.fit(train_images_76,train_images_76, batch_size=64, epochs=80)


Epoch 1/80
1604/1604 [==============================] - 1s - loss: 181.7060     
Epoch 2/80
1604/1604 [==============================] - 1s - loss: 181.6954     
Epoch 3/80
1604/1604 [==============================] - 1s - loss: 182.2254     
Epoch 4/80
1604/1604 [==============================] - 1s - loss: 183.2608     
Epoch 5/80
1604/1604 [==============================] - 1s - loss: 183.9101     
Epoch 6/80
1604/1604 [==============================] - 1s - loss: 185.2260     
Epoch 7/80
1604/1604 [==============================] - 1s - loss: 181.7332     
Epoch 8/80
1604/1604 [==============================] - 1s - loss: 181.0387     
Epoch 9/80
1604/1604 [==============================] - 1s - loss: 182.7954     
Epoch 10/80
1604/1604 [==============================] - 1s - loss: 180.8999     
Epoch 11/80
1604/1604 [==============================] - 1s - loss: 180.3809     
Epoch 12/80
1604/1604 [==============================] - 1s - loss: 181.1455     
Epoch 13/80
1604/1604 [==

In [10]:
encoder = Model(input = inputs, output = encoded_flat)
#encoder.predict(train_images_76)[0]
pred_full_train = encoder.predict(full_train)
pred_train =  encoder.predict(train_images_76)
pred_test = encoder.predict(test_images_76)

/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fl...)`
  """Entry point for launching an IPython kernel.


In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(pred_full_train)

pred_train_scaled = scaler.transform(pred_train)
pred_test_scaled = scaler.transform(pred_test)


In [12]:
pred_train_scaled.shape

(1604, 648)

In [34]:
from sklearn import linear_model
logistic = linear_model.LogisticRegression(C=1e5)

logistic.fit(pred_train_scaled,y_train)

/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [35]:
p = logistic.predict_proba(pred_test_scaled)

sub = pd.DataFrame()
sub['id']  = test_df['id']
sub['is_iceberg'] = p[:,1]
sub.to_csv('sub/autoenc_logistic.csv', index = False)
p.mean()

0.5

In [41]:
p[:,1].round(3).mean()

0.84597293447293453

In [89]:
from keras.layers import Dropout
inputs_ = Input(shape=(648,))
#a = Dropout(0.3)(inputs_)

#a = Dense(int(648/2), activation='relu')(a)
#a = Dropout(0.4)(a)

a = Dense(int(200/2), activation='relu')(inputs_)
#a = Dropout(0.4)(a)
# And finally we add the main logistic regression layer
main_output_ = Dense(1, activation='sigmoid', name='main_output')(a)

model2 = Model(input=inputs_, output=main_output_)
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#score = model.evaluate(X_test, y_test, batch_size=128)


/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ma...)`
  del sys.path[0]


In [95]:
#X_train2, X_test2 = train_test_split(pred_train_scaled, test_size=0.33, random_state=42)
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=4)

model2.fit(pred_train_scaled,y_train, validation_split=0.3, batch_size=64, epochs=200, callbacks=[early_stopping_monitor])

Train on 1122 samples, validate on 482 samples
Epoch 1/200
1122/1122 [==============================] - 0s - loss: 0.3041 - acc: 0.8717 - val_loss: 0.3953 - val_acc: 0.8029
Epoch 2/200
1122/1122 [==============================] - 0s - loss: 0.3042 - acc: 0.8681 - val_loss: 0.4179 - val_acc: 0.8029
Epoch 3/200
1122/1122 [==============================] - 0s - loss: 0.2965 - acc: 0.8725 - val_loss: 0.3792 - val_acc: 0.8154
Epoch 4/200
1122/1122 [==============================] - 0s - loss: 0.3016 - acc: 0.8565 - val_loss: 0.3757 - val_acc: 0.8154
Epoch 5/200
1122/1122 [==============================] - 0s - loss: 0.3028 - acc: 0.8645 - val_loss: 0.3757 - val_acc: 0.8133
Epoch 6/200
1122/1122 [==============================] - 0s - loss: 0.3069 - acc: 0.8601 - val_loss: 0.5181 - val_acc: 0.7220
Epoch 7/200
1122/1122 [==============================] - 0s - loss: 0.3115 - acc: 0.8503 - val_loss: 0.5054 - val_acc: 0.7365
Epoch 8/200
1122/1122 [==============================] - 0s - loss: 0.3

In [96]:
p = model2.predict(pred_test_scaled)

In [98]:
sub = pd.DataFrame()
sub['id']  = test_df['id']
sub['is_iceberg'] = p
sub.to_csv('sub/autoenc.csv', index = False)

In [97]:
p.mean()

0.12991056

In [87]:
pred = autoencoder.predict(train_images_76)
pred_res = pred.reshape(-1,72,72,3)

In [ ]:
import matplotlib.pyplot as plt
plt.matshow(pred_res[50,:,:,0])

In [ ]:
plt.matshow(train_images_76[50,:,:,0])